In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dill
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


if os.path.exists('final_project.db'):
    dill.load_session('final_project.db')

In [6]:
# convert 2019 objects to correct types
df_2019['datetime'] = pd.to_datetime(df_2019['datetime'], format='%Y-%m-%d')
df_2019['gender'] = df_2019['gender'].astype('category')
df_2019['age_group'] = df_2019['age_group'].astype('category')
df_2019['country'] = df_2019['country'].astype('category')
df_2019['major'] = df_2019['major'].astype('category')
df_2019.drop(columns=['Unnamed: 0'], inplace=True)

# convert 2020 objects to correct types
df_2020['datetime'] = pd.to_datetime(df_2020['datetime'], format='%Y-%m-%d')
df_2020['gender'] = df_2020['gender'].astype('category')
df_2020['age_group'] = df_2020['age_group'].astype('category')
df_2020['country'] = df_2020['country'].astype('category')
df_2020['major'] = df_2020['major'].astype('category')
df_2020.drop(columns=['Unnamed: 0'], inplace=True)

display(df_2019.head())
display(df_2020.head())

,datetime,athlete,distance,duration,gender,age_group,country,major
0,2019-01-01,0,0.00,0.00,F,18 - 34,United States,CHICAGO 2019
1,2019-01-01,1,5.27,30.20,M,35 - 54,Germany,BERLIN 2016
2,2019-01-01,2,9.30,98.00,M,35 - 54,United Kingdom,"LONDON 2018,LONDON 2019"
3,2019-01-01,3,103.13,453.40,M,18 - 34,United Kingdom,LONDON 2017
4,2019-01-01,4,34.67,185.65,M,35 - 54,United States,BOSTON 2017


,datetime,athlete,distance,duration,gender,age_group,country,major
0,2020-01-01,0,0.00,0.000000,F,18 - 34,United States,CHICAGO 2019
1,2020-01-01,1,70.33,394.200000,M,35 - 54,Germany,BERLIN 2016
2,2020-01-01,2,14.65,79.066667,M,35 - 54,United Kingdom,"LONDON 2018,LONDON 2019"
3,2020-01-01,3,41.41,195.666667,M,18 - 34,United Kingdom,LONDON 2017
4,2020-01-01,4,41.34,209.100000,M,35 - 54,United States,BOSTON 2017


In [8]:
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1893424 entries, 0 to 1893423
Data columns (total 8 columns):
 #   Column     Dtype         
---  ------     -----         
 0   datetime   datetime64[ns]
 1   athlete    int64         
 2   distance   float64       
 3   duration   float64       
 4   gender     category      
 5   age_group  category      
 6   country    category      
 7   major      category      
dtypes: category(4), datetime64[ns](1), float64(2), int64(1)
memory usage: 68.7 MB


In [9]:
X_2019 = df_2019.drop(columns=['distance'])
y_2019 = df_2019['distance']
X_2019_train, X_2019_test, y_2019_train, y_2019_test = train_test_split(X_2019, y_2019, test_size=0.3)

In [ ]:
# Add new columns where, 52:
# Rows are individual athletes
# Columns are the mileage for x week

df_2019.groupby(['athlete'])['datetime'].value_counts()


athlete  datetime  
0        2019-01-01    1
         2019-01-08    1
         2019-01-15    1
         2019-01-22    1
         2019-01-29    1
                      ..
37598    2019-11-26    1
         2019-12-03    1
         2019-12-10    1
         2019-12-17    1
         2019-12-24    1
Name: count, Length: 1893424, dtype: int64

In [134]:
df_2019_new = df_2019.pivot_table(
    index='athlete',
    columns='datetime',
    values='distance',
    aggfunc='sum',
    fill_value=0
)
df_2019_new.columns = [f'week_{i}' for i in range(df_2019_new.shape[1])]
df_2019_new = df_2019_new.reset_index()
mask = ~df_2019['athlete'].duplicated()
df_2019_new['age_group'] = df_2019[mask]['age_group']
df_2019_new['country'] = df_2019[mask]['country']
df_2019_new['gender'] = df_2019[mask]['gender']
df_2019_new['major'] = df_2019[mask]['major']
df_2019_new

,athlete,week_0,week_1,week_2,week_3,week_4,week_5,week_6,week_7,week_8,...,week_46,week_47,week_48,week_49,week_50,week_51,age_group,country,gender,major
0,0,0.00,0.000,0.00,0.000,0.000,0.00,0.000,0.00,0.000,...,0.000,0.00,0.00,0.000,0.000,0.000000,18 - 34,United States,F,CHICAGO 2019
1,1,5.27,59.860,55.99,58.500,58.180,51.59,63.710,62.04,52.480,...,10.770,30.30,39.47,50.439,62.530,70.280000,35 - 54,Germany,M,BERLIN 2016
2,2,9.30,30.820,10.01,54.340,37.099,58.28,61.690,61.16,71.319,...,9.380,22.90,25.54,14.490,0.000,8.163750,35 - 54,United Kingdom,M,"LONDON 2018,LONDON 2019"
3,3,103.13,93.100,87.40,97.840,54.870,9.76,87.260,4.88,41.060,...,40.300,42.02,42.15,45.710,21.120,63.516250,18 - 34,United Kingdom,M,LONDON 2017
4,4,34.67,0.000,30.51,38.680,0.000,38.30,0.000,8.66,10.160,...,32.210,41.05,44.31,5.110,3.830,9.966250,35 - 54,United States,M,BOSTON 2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36407,37594,168.05,113.140,163.52,161.509,163.320,123.18,66.189,88.89,149.859,...,34.870,48.25,38.37,40.410,48.029,12.835375,18 - 34,United Kingdom,M,BERLIN 2017
36408,37595,79.81,114.879,113.51,91.680,128.270,136.32,121.530,127.39,134.540,...,129.499,135.63,136.03,135.970,116.689,151.725000,18 - 34,United States,M,"BERLIN 2019,NEW YORK 2015"
36409,37596,118.89,111.070,117.22,136.400,134.308,136.25,118.340,90.93,92.400,...,29.710,68.91,48.00,49.910,8.060,35.770000,18 - 34,United States,M,BOSTON 2017
36410,37597,28.67,54.410,49.88,41.220,48.930,50.09,75.060,23.43,72.260,...,53.160,15.53,37.90,24.670,31.530,31.937500,18 - 34,United States,F,BOSTON 2015
